In [1]:
import numpy as np
import pandas as pd

### Task is to first scrape Wikipedia page to set up DataFrame of Toronto Postal Codes, Boroughs and Neighbourhoods

### Reference for Instructions on assignment for this task

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

To create the above dataframe:
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 
1. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
1. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
1. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
1. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [2]:
# obtain relevant HTML file from Wikipedia
from bs4 import BeautifulSoup
import requests 

toronto_wiki_html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_html_doc = requests.get(toronto_wiki_html)
soup = BeautifulSoup(toronto_html_doc.text, 'html.parser')

In [3]:
data = []
table = soup.find('table', attrs={'class': 'wikitable sortable'}) # extract relevant table from BeautifulSoup
table_body = table.find('tbody')

# extract all rows in table and assign to data list
rows = table_body.find_all('tr') 
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols])
    
toronto_df = pd.DataFrame(data[1:]) # convert data to DataFrame
toronto_df.columns = ['PostalCode', 'Borough', 'Neighbourhood'] # rename columns
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned'].reset_index(drop=True) # drop rows with Borough = 'Not assigned'

index_no_assigned_neighbourhood = toronto_df[toronto_df['Neighbourhood'] == 'Not assigned'].index # find rows with Neighbourhood = 'Not assigned'
toronto_df['Neighbourhood'][index_no_assigned_neighbourhood] = toronto_df['Borough'][index_no_assigned_neighbourhood] # assign Neighbourhood = Borough

toronto_df_temp = pd.DataFrame(toronto_df.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))).reset_index() # groupby postal code and concatenate Neighbourhoods

toronto_df = toronto_df_temp.join(toronto_df[['PostalCode', 'Borough']].drop_duplicates(subset='PostalCode').set_index('PostalCode'), on='PostalCode') # join Borough column back

toronto_df = toronto_df[['PostalCode', 'Borough', 'Neighbourhood']] # rearrange columns to right order

print('Number of rows = ', toronto_df.shape[0])
toronto_df.head(10) # show first 10 rows


Number of rows =  103


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
